By leveraging the power of LangChain, SQL Agents, and OpenAI's Large Language Models (LLMs) like ChatGPT, we can create applications that enable users to query databases using natural language.

In [1]:
# !pip install langchain openai pymysql 
# !pip install google-search-results

In [2]:
import openai
import os 
import time
import warnings
warnings.filterwarnings('ignore')

os.environ['OPENAI_API_KEY'] = ""
os.environ["SERPAPI_API_KEY"] = ""

In [4]:
from langchain.llms import OpenAI
from langchain.agents import AgentType , load_tools , initialize_agent

In [5]:
llm = OpenAI(temperature = 0 , model = 'text-davinci-003')

tools = load_tools(['serpapi' , 'llm-math'] , llm = llm)
agent = initialize_agent(tools = tools , llm = llm , agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION , verbose = True)

In [8]:
try:
    res = agent.run("Who is Sachin Tendulkar? What is his current age raised to the power 2?")
except openai.OpenAIError as e:
    print(f"An error occurred: {e}")

In [7]:
res

'Sachin Ramesh Tendulkar, BR AO is 50 years old and his age raised to the power 2 is 2500.'

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.sql_database import SQLDatabase
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit

In [14]:
db_user = ""
db_password = ""
db_host = ""
port = ""
db_name = "classicmodels"

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{port}/{db_name}")

In [15]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True
)

In [20]:
res = agent_executor.run("List the top 3 countries with the highest number of orders")

In [17]:
res

'USA, France, Spain'

In [21]:
res = agent_executor.run("Find the top 5 products with the highest total sales revenue")

In [22]:
res

'The top 5 products with the highest total sales revenue are:\n\n1. 1995 Honda Civic - $917,493.08\n2. 1952 Alpine Renault 1300 - $720,126.90\n3. 1962 LanciaA Delta 16V - $702,325.22\n4. 1968 Dodge Charger - $685,684.68\n5. 1976 Ford Gran Torino - $670,743.23'